# Aplicaciones de Transformers en Procesamiento de Lenguaje Natural

Este cuaderno explora distintas aplicaciones de los modelos Transformers en tareas de procesamiento de lenguaje natural (PLN), utilizando ejemplos prácticos y explicaciones teóricas.

---

## ¿Qué es la arquitectura Transformer?

La **arquitectura Transformer** es un tipo de red neuronal propuesta por Vaswani et al. en 2017. Su principal innovación es el mecanismo de *atención*, que permite al modelo enfocarse en diferentes partes de la entrada para procesar información contextual de manera eficiente. Los Transformers reemplazaron a modelos anteriores como LSTM y GRU en muchas tareas de PLN debido a su capacidad para manejar secuencias largas y aprender dependencias complejas.

---

## ¿Qué es la librería Transformers de Hugging Face?

La **librería Transformers de Hugging Face** es una biblioteca de Python que proporciona implementaciones preentrenadas de modelos basados en la arquitectura Transformer (como BERT, GPT, T5, etc.) para tareas de PLN. Permite cargar modelos fácilmente y utilizarlos en tareas como clasificación, traducción, resumen, etc., sin necesidad de entrenarlos desde cero. Podes explorar los modelos disponibles en la [página de modelos de Hugging Face](https://huggingface.co/models).

---

## ¿Qué es un pipeline en Hugging Face?

Un **pipeline** en Hugging Face es una interfaz de alto nivel que simplifica el uso de modelos preentrenados para tareas específicas. Permite ejecutar tareas como clasificación de texto, reconocimiento de entidades, traducción, etc., con solo unas pocas líneas de código, ocultando detalles complejos de preprocesamiento y postprocesamiento.


## 1. Definir el texto de ejemplo

A continuación, se presenta un texto de ejemplo que será utilizado en las distintas aplicaciones de modelos Transformers. Este texto simula una queja de un cliente sobre un pedido equivocado, lo que nos permitirá explorar tareas como clasificación, reconocimiento de entidades, respuesta a preguntas, resumen, traducción y generación de texto.

In [ ]:
texto = """Querido MercadoLibre, la semana pasada pedí una figura de acción de Optimus Prime desde su tienda online.
Para mi sorpresa, cuando abrí el paquete, descubrí horrorizado que me habían enviado una figura de Megatron.
Como fan de los Autobots, espero que entiendan mi decepción. Solicito un cambio urgente del producto."""

## 2. Clasificación de texto con Transformers

La **clasificación de texto** es una tarea fundamental en PLN que consiste en asignar una o varias etiquetas a un texto, como por ejemplo identificar el sentimiento (positivo, negativo, neutro) o la intención del mensaje. Los modelos Transformers, gracias a su capacidad para comprender el contexto, demostraron un rendimiento sobresaliente en esta tarea.

Utilizaremos el pipeline `text-classification` de la librería 🤗 Transformers para analizar el texto de ejemplo y visualizar el resultado de la clasificación.

In [ ]:
from transformers import pipeline
# Modelo de clasificación de texto (sentimiento) en español
# Puedes buscar otros modelos en https://huggingface.co/models
classifier = pipeline("text-classification", model="pysentimiento/robertuito-sentiment-analysis")
import pandas as pd
outputs = classifier(texto)
pd.DataFrame(outputs)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/925 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/435M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

Device set to use cpu


,label,score
0,NEG,0.913063


## 3. Reconocimiento de entidades nombradas (NER)

El **Reconocimiento de Entidades Nombradas** (NER, por sus siglas en inglés) es la tarea de identificar y clasificar automáticamente nombres propios, lugares, organizaciones y otras entidades relevantes dentro de un texto. Los Transformers permiten realizar NER de manera eficiente y precisa.

A continuación, aplicamos el pipeline `ner` al texto de ejemplo para extraer las entidades nombradas presentes.

In [ ]:
# Modelo de reconocimiento de entidades nombradas (NER) en español
# Puedes buscar otros modelos en https://huggingface.co/models
ner_tagger = pipeline("ner", model="mrm8488/bert-spanish-cased-finetuned-ner", aggregation_strategy="simple")
outputs = ner_tagger(texto)
pd.DataFrame(outputs)

config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

Some weights of the model checkpoint at mrm8488/bert-spanish-cased-finetuned-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


,entity_group,score,word,start,end
0,ORG,0.756585,MercadoLibre,8,20
1,MISC,0.829108,Optimus Prime,68,81
2,MISC,0.763651,Megatron,205,213
3,MISC,0.954693,Autobot,231,238


## 4. Respuesta a preguntas basada en contexto

La tarea de **respuesta a preguntas** (Question Answering) consiste en responder preguntas específicas utilizando un contexto dado. Los modelos Transformers pueden comprender el texto y extraer la información relevante para responder de manera precisa.

En el siguiente ejemplo, preguntamos al modelo qué desea el cliente, utilizando el pipeline `question-answering`.

In [ ]:
# Modelo de respuesta a preguntas en español
# Puedes buscar otros modelos en https://huggingface.co/models
reader = pipeline("question-answering", model="PlanTL-GOB-ES/roberta-large-bne-sqac")
question = "¿Qué quiere el cliente?"
outputs = reader(question=question, context=texto)
pd.DataFrame([outputs])

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.07k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/858k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/516k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.48M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Device set to use cpu


,score,start,end,answer
0,0.999948,285,315,un cambio urgente del producto


## 5. Resumen automático de texto

El **resumen automático** permite condensar la información principal de un texto extenso en una versión más corta, manteniendo el significado esencial. Los Transformers han revolucionado esta tarea gracias a su capacidad de comprensión contextual.

A continuación, utilizamos el pipeline `summarization` para obtener un resumen del texto de ejemplo.

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# Puedes buscar otros modelos en https://huggingface.co/models
model_name = "csebuetnlp/mT5_multilingual_XLSum"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
resumidor = pipeline("summarization", model=model, tokenizer=tokenizer)
resumen = resumidor(texto, max_length=80, min_length=20, do_sample=False)
print(resumen[0]['summary_text'])

tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/730 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

Device set to use cpu


Hace unos meses, un paquete de acciones de la tienda online Amazon me envió una figura de Megatron.


## 6. Traducción automática del español a inglés

La **traducción automática** es una de las aplicaciones más destacadas de los Transformers, permitiendo traducir textos entre diferentes idiomas con alta calidad. Utilizaremos el pipeline `translation_es_to_en` para traducir el texto de ejemplo del español al inglés.

In [ ]:
# Modelo de traducción de español a inglés
# Puedes buscar otros modelos en https://huggingface.co/models
translator = pipeline("translation_es_to_en", model="Helsinki-NLP/opus-mt-es-en")
outputs = translator(texto)
print(outputs[0]['translation_text'])

config.json:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


...

source.spm:   0%|          | 0.00/826k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.59M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:177: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cpu


Dear MarketFree, last week I asked for an action figure of Optimus Prime from his online store. To my surprise, when I opened the package, I discovered horrified that I had been sent a figure of Megatron. As a fan of Autobots, I hope you understand my disappointment. I request an urgent change of product.


## 7. Generación automática de texto (respuesta de servicio al cliente)

La **generación automática de texto** permite crear o continuar textos de manera coherente a partir de un prompt inicial. Esta capacidad es útil, por ejemplo, para redactar respuestas automáticas en atención al cliente.

En el siguiente ejemplo, generamos una posible respuesta de servicio al cliente utilizando el pipeline `text-generation`.

In [ ]:
# Modelo de generación de texto en español
# Puedes buscar otros modelos en https://huggingface.co/models
generator = pipeline("text-generation", model="datificate/gpt2-small-spanish")
respuesta_inicial = "Estimado cliente, lamentamos mucho lo ocurrido con su pedido. "
prompt = texto + "\n\nRespuesta del servicio al cliente:\n" + respuesta_inicial
outputs = generator(
    prompt,
    max_new_tokens=150,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.9,
    repetition_penalty=1.3,
    eos_token_id=50256
)
print(outputs[0]['generated_text'])

config.json:   0%|          | 0.00/817 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/510M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/510M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/620 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/850k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/508k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

Device set to use cpu


Querido MercadoLibre, la semana pasada pedí una figura de acción de Optimus Prime desde su tienda online.
Para mi sorpresa, cuando abrí el paquete, descubrí horrorizado que me habían enviado una figura de Megatron.
Como fan de los Autobots, espero que entiendan mi decepción. Solicito un cambio urgente del producto.

Respuesta del servicio al cliente:
Estimado cliente, lamentamos mucho lo ocurrido con su pedido.  No quiero que esto vuelva a suceder y no dejé indiferente de verme como si este se hubiese convertido así después de que pensara que sería bueno.
Reactivando mis pensamientos sobre esta noticia es difícil.
El día 26 de septiembre, fue anunciado que era el estreno oficial para PlayStation 4 y Xbox One titulado "Transformers", junto con las nuevas entregas de Transformer 2 y Transformers 3; por lo tanto, fue recibido positivamente entre los fanáticos más jóvenes e incluso adultos ya existentes durante todo ese tiempo (ya sea recién llegados o nuevos).
La primera entrega de "trans

---

## Ejercicios autónomos (40 minutos)

A continuación, te proponemos ejercicios prácticos para que pongas en práctica lo aprendido sobre la arquitectura Transformer, la librería Transformers de Hugging Face y el uso de pipelines en tareas de PLN.

**Recorda:** Para cada ejercicio, explora la [página de modelos de Hugging Face](https://huggingface.co/models) y busca modelos que sean apropiados para español, o incluso para el contexto argentino si los encontras.

### 1. Clasificación de textos propios
- Escribi dos textos breves (pueden ser opiniones, quejas o comentarios sobre servicios o productos en Argentina, por ejemplo sobre colectivos, supermercados, bancos, etc.).
- Busca en Hugging Face un modelo de clasificación de texto en español y usalo con el pipeline `text-classification` para analizar el sentimiento de cada texto.
- ¿El resultado coincide con tu expectativa? Explica brevemente.

### 2. Reconocimiento de entidades en noticias argentinas
- Busca un párrafo de una noticia reciente de un medio argentino (por ejemplo, Clarín, La Nación, Página/12).
- Busca en Hugging Face un modelo de NER para español y aplícalo para identificar entidades nombradas.
- Enumera las entidades encontradas y clasifícalas (persona, organización, lugar, etc.).

### 3. Respuesta a preguntas personalizada
- Escribi un pequeño texto narrativo (4-5 líneas) sobre una situación cotidiana en Buenos Aires (por ejemplo, un reclamo en una oficina pública, una experiencia en el subte, etc.).
- Formula dos preguntas sobre el texto y utiliza un modelo de `question-answering` de Hugging Face para responderlas.
- ¿Las respuestas son correctas y precisas? ¿Por qué?

### 4. Resumen de un texto propio
- Elegí un texto más largo (puede ser un fragmento de Wikipedia sobre Argentina, una noticia, etc.).
- Busca un modelo de resumen en español en Hugging Face y úsalo con el pipeline `summarization`.
- Compara el resumen generado con el texto original: ¿qué información se perdió y cuál se mantuvo?

### 5. Traducción y análisis
- Escribi un texto breve en español rioplatense y tradúcilo al inglés usando un modelo de traducción de Hugging Face.
- Luego, traduci el resultado nuevamente al español (podes usar otro pipeline o Google Translate).
- ¿El texto final es igual al original? ¿Qué diferencias encontras?

### 6. Generación creativa de texto
- Escribe el inicio de una historia o una pregunta abierta relacionada con Buenos Aires.
- Busca un modelo de generación de texto en español en Hugging Face y úsalo con el pipeline `text-generation` para continuar el texto.
- Analiza la coherencia y creatividad de la respuesta generada.

---

**Tip:** Para cada ejercicio, explora la [página de modelos de Hugging Face](https://huggingface.co/models) y lee la descripción de los modelos antes de usarlos. Si tienes dudas, consulta la documentación oficial de [Transformers](https://huggingface.co/docs/transformers/index).

In [ ]:
from transformers import pipeline

textos = [
    "Otra vez el colectivo llegó con 25 minutos de demora. Ya ni sé para qué están los horarios, una falta de respeto total.",
    "Fui al supermercado nuevo del barrio y me encantó. Todo estaba limpio, bien organizado y la atención fue excelente."
]

clasificador = pipeline("text-classification", model="pysentimiento/robertuito-sentiment-analysis")
resultados = clasificador(textos)

for i, resultado in enumerate(resultados):
    print(f"Texto {i+1}:")
    print(f"  → Etiqueta: {resultado['label']}")
    print(f"  → Confianza: {resultado['score']:.2f}")

Device set to use cpu


Texto 1:
  → Etiqueta: NEG
  → Confianza: 0.98
Texto 2:
  → Etiqueta: POS
  → Confianza: 0.98


In [ ]:
import pandas as pd

# Texto de ejemplo basado en noticia argentina
texto = (
    "El presidente Javier Milei anunció esta mañana en Casa Rosada un nuevo paquete de medidas económicas "
    "junto al ministro de Economía, Luis Caputo. La conferencia fue transmitida en vivo por la Televisión Pública "
    "y seguida de cerca por analistas en Buenos Aires."
)

# Cargar el modelo NER en español
ner = pipeline("ner", model="mrm8488/bert-spanish-cased-finetuned-ner", aggregation_strategy="simple")

# Aplicar el modelo al texto
entidades = ner(texto)

# Mostrar resultados en una tabla
pd.DataFrame(entidades)

Some weights of the model checkpoint at mrm8488/bert-spanish-cased-finetuned-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


,entity_group,score,word,start,end
0,PER,0.998498,Javier Milei,14,26
1,LOC,0.999310,Casa Rosada,50,61
2,MISC,0.999771,Economía,122,130
3,PER,0.999750,Luis Caputo,132,143
4,ORG,0.999277,Televisión Pública,191,209
5,LOC,0.999503,Buenos Aires,246,258


In [ ]:
# Texto narrativo
contexto = (
    "Ayer fui a la oficina de ANSES de la calle Córdoba para hacer un reclamo por un error en mi jubilación. "
    "La atención fue lenta, pero finalmente me atendió una señora muy amable que resolvió mi problema."
)

# Cargar modelo de respuesta a preguntas en español
qa = pipeline("question-answering", model="PlanTL-GOB-ES/roberta-large-bne-sqac")

# Preguntas
preguntas = [
    "¿Dónde fue el reclamo?",
    "¿Cómo fue la atención recibida?",
    "¿Quién atendió el reclamo?"
]

# Obtener respuestas
respuestas = [qa(question=p, context=contexto) for p in preguntas]

# Mostrar resultados
pd.DataFrame({
    "Pregunta": preguntas,
    "Respuesta": [r["answer"] for r in respuestas],
    "Confianza": [r["score"] for r in respuestas]
})

Device set to use cpu


,Pregunta,Respuesta,Confianza
0,¿Dónde fue el reclamo?,oficina de ANSES de la calle Córdoba,0.883641
1,¿Cómo fue la atención recibida?,lenta,0.999893
2,¿Quién atendió el reclamo?,una señora muy amable,0.999952


In [ ]:
# Texto original
texto = (
    "Argentina es un país soberano de América del Sur, ubicado en el extremo sur y sudeste del subcontinente. "
    "Limita al norte con Bolivia y Paraguay, al noreste con Brasil, al este con Uruguay y el océano Atlántico Sur, "
    "y al sur y oeste con Chile. Con una superficie de más de 2.7 millones de km², es el octavo país más grande del mundo. "
    "Su capital es Buenos Aires, una ciudad cosmopolita y uno de los principales centros culturales y económicos del continente. "
    "Argentina es una república federal compuesta por 23 provincias y una ciudad autónoma. "
    "Su economía es una de las más grandes de América Latina, con sectores fuertes como la agricultura, la ganadería, la industria y los servicios."
)

# Cargar modelo de resumen
model_name = "csebuetnlp/mT5_multilingual_XLSum"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

resumidor = pipeline("summarization", model=model, tokenizer=tokenizer)

# Generar resumen
resumen = resumidor(texto, max_length=80, min_length=20, do_sample=False)
print("Resumen generado:")
print(resumen[0]['summary_text'])

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Device set to use cpu


Resumen generado:
Argentina es un país soberano de América del Sur, ubicado en el extremo sur y sudeste del subcontinente.


In [ ]:
# Texto original
texto_original = (
    "Che, ayer fui al laburo en bondi y me clavé un cafecito antes de arrancar. "
    "Estaba hecho percha, pero por suerte el jefe no dijo nada."
)

# Traducción español → inglés
traductor_es_en = pipeline("translation", model="Helsinki-NLP/opus-mt-es-en")
texto_en = traductor_es_en(texto_original)[0]['translation_text']

# Traducción inglés → español
traductor_en_es = pipeline("translation", model="Helsinki-NLP/opus-mt-en-es")
texto_es_retraducido = traductor_en_es(texto_en)[0]['translation_text']

# Mostrar resultados
print("🔹 Español original:\n", texto_original)
print("\n🔹 Traducción al inglés:\n", texto_en)
print("\n🔹 Retraducción al español:\n", texto_es_retraducido)

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:177: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cpu


config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/312M [00:00<?, ?B/s]

...

source.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/826k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.59M [00:00<?, ?B/s]

Device set to use cpu


🔹 Español original:
 Che, ayer fui al laburo en bondi y me clavé un cafecito antes de arrancar. Estaba hecho percha, pero por suerte el jefe no dijo nada.

🔹 Traducción al inglés:
 Che, yesterday I went to the job in bondi and stuck a coffee before I started. I was hanged, but luckily the boss didn't say anything.

🔹 Retraducción al español:
 Che, ayer fui al trabajo en Bondi y me puse un café antes de empezar. Me colgaron, pero por suerte el jefe no dijo nada.


In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

# Inicio de historia
prompt = (
    "Eran las dos de la madrugada y la lluvia no paraba en Buenos Aires. "
    "Martín caminaba por Avenida Corrientes cuando algo llamó su atención desde una vieja librería iluminada..."
)

# Cargar modelo de generación (ligero para Colab)
model_name = "tiiuae/falcon-rw-1b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Crear pipeline de generación
generador = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Generar texto
output = generador(prompt, max_new_tokens=100, do_sample=True, top_k=50, temperature=0.8)

# Mostrar resultado
print("📝 Historia completada:")
print(output[0]["generated_text"])

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


📝 Historia completada:
Eran las dos de la madrugada y la lluvia no paraba en Buenos Aires. Martín caminaba por Avenida Corrientes cuando algo llamó su atención desde una vieja librería iluminada...
It's the middle of the night in Buenos Aires when a woman's attention is drawn to a doorway in an old library. A voice calls to her.
She thinks, "I've heard of this voice before."
She turns to go.
But it's too late.
She stops and turns back.
In her way, a man is coming through the door, heading the opposite direction. He turns toward her and she recognizes him.
She knows what he looks like.
